## Скрипт предсказывает тему записи, на основе заголовка и текста новости. Темы сгруппированы в 4 группы. Достигаемые значения метрики F1 - 85.3%. 
## Для предсказани классов используется обученная сеть прямого распространения. На вход сети подается выходной вектор ELMO (сформированный на основании оценки заголовка) и результат парсинга текста на наличие географических названий. 
## Результат классифицирует тексты на 22 темы. F1-69% После чего темы объеденяются в 4 группы и значения метрики увеличиваются до 85%

In [1]:
import os
import numpy as np
import pandas as pd
pd.options.display.max_rows = 120
import torch
import torch.nn as nn
import torch.optim as optim
import re
import pickle
import pymorphy2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn.decomposition import PCA 
from transformers import BertModel, BertTokenizer, BertConfig, AutoTokenizer
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import warnings
warnings.simplefilter(action='ignore')

%matplotlib inline

In [2]:
morf = pymorphy2.MorphAnalyzer()

## Подготовка данных

In [3]:
#загружаем заранее скачанный датасет
df0=pd.read_csv('lenta-ru-news.csv',low_memory=False)

In [5]:
# удаляем данные с незаполненной темой так как там предсказывать нечего
df=df0[pd.isnull(df0.topic)==False]
df=df[pd.isnull(df.text)==False]
df=df[pd.isnull(df.title)==False]
df=df.reset_index().drop(columns='index')

In [6]:
df.shape

(738968, 6)

In [9]:
# перемешиваем и отбираем первые 100 тысяч
# на большем количестве результат будет не хуже, а время работы - существенно дольше
# 100 тыс - обрабатываются за 3.5 часа соответственно 700 тыс - за 24 часа.

df1=shuffle(df).reset_index().drop(columns='index')
df=df1.loc[:99999,:]

In [36]:
print(df.topic.value_counts().shape[0])
top_ls=list(df.topic.value_counts().index)
top_ls

22


['Россия',
 'Мир',
 'Экономика',
 'Спорт',
 'Бывший СССР',
 'Культура',
 'Наука и техника',
 'Интернет и СМИ',
 'Из жизни',
 'Дом',
 'Силовые структуры',
 'Ценности',
 'Бизнес',
 'Путешествия',
 '69-я параллель',
 'Крым',
 'Культпросвет ',
 'Легпром',
 'Библиотека',
 'МедНовости',
 'Оружие',
 'Сочи']

In [36]:
df['top_id']=df.topic.apply(lambda x: top_ls.index(x))
df['tit_len']=df.title.apply(lambda x: len(x.split()))

In [37]:
df.head()

,url,title,text,topic,tags,date,top_id,tit_len
0,https://lenta.ru/news/2013/08/23/rating/,Профессия рыбака перестала быть самой опасной ...,Впервые за последние несколько лет профессия р...,Экономика,Деньги,2013/08/23,2,8
1,https://lenta.ru/news/2009/11/12/cobrand/,Антимонопольщики России проверят законность ко...,Федеральная антимонопольная служба России запр...,Экономика,Все,2009/11/12,2,6
2,https://lenta.ru/news/2015/09/05/miting/,В Донецке прошел митинг в поддержку бывшего гл...,В центре Донецка несколько десятков митингующи...,Бывший СССР,Украина,2015/09/05,4,10
3,https://lenta.ru/news/2013/07/04/liberty/,Статую Свободы открыли после ремонта,В Нью-Йорке 4 июля открыли доступ посетителей ...,Мир,Общество,2013/07/04,1,5
4,https://lenta.ru/news/2014/04/27/georgia/,В Грузии начали вербовать наемников для отправ...,Бывший начальник личной охраны экс-президента ...,Бывший СССР,Украина,2014/04/27,4,9


## Поскольку основые темы называются Россия и Мир, подсчитаем сколько раз в тексте встречаются географические названия из России, СНГ и остального мира  

In [6]:
# загружаем справочник географических названий
df_geo=pd.read_csv('geographic_spr.csv')

In [7]:
df_geo.head(2)

,Unnamed: 0,country,city,region
0,0,Россия,Москва,Москва и Московская обл.
1,1,Россия,Абрамцево,Москва и Московская обл.


In [8]:
# функция возвращающая к какой части света относится данное название
def match_geo(wrd):
    
    wrd = wrd[0].upper()+wrd[1:]
    cantr=''
    if df_geo[df_geo.city==wrd].country.shape[0]!=0:
        cantr = df_geo[df_geo.city==wrd].country.iloc[0]
    if df_geo[df_geo.region==wrd].country.shape[0]!=0:
        cantr = df_geo[df_geo.region==wrd].country.iloc[0]
    if df_geo[df_geo.country==wrd].country.shape[0]!=0:
        cantr = df_geo[df_geo.country==wrd].country.iloc[0]
        
    if cantr=='':
        return 0
    elif cantr == 'Россия':
        return 1
    elif cantr in ['Беларусь','Украина','Молдова','Литва','Латвия','Эстония',
                  'Грузия','Армения','Азербайджан','Туркмения','Казахстан','Узбекистан',
                  'Киргызстан','Таджикистан']:
        return 2
     
    return 3
    

In [9]:
# Функция, выбирает из текста имена собственные и анализирует их на предмет географии
def geo_text(txt):
    pattern='[А-Я]'+'\w*'
    x=0
    y=0
    z=0
    m=0
    
    for wrd in re.findall(pattern,txt):
        norm_wrd=morf.parse(wrd)[0].normal_form

        answ=match_geo(norm_wrd)
        if answ==1:
            x+=1
        elif answ==2:
            y+=1
        elif answ==3:
            z+=1
        else:
            m+=1
            
    return x,y,z,m

# Создаем тензор с гео результатами

In [10]:
%%time
geo=torch.zeros(df.shape[0],4)
for i in range(df.shape[0]):
    geo[i,:]=torch.tensor(geo_text(df.text[i]))
    if i%5000==0:
        print(i)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
Wall time: 2h 22min 24s


In [12]:
torch.save(geo,'geo_tz_100000.pt')

## Загружаем ELMo от ruwikiruscorpora_tokens_elmo_1024_2019
## https://rusvectores.org/ru/models/
##  НКРЯ и Википедия за декабрь 2018 (токены)


In [23]:
from allennlp.modules.elmo import Elmo, batch_to_ids
from scipy.spatial.distance import cosine

In [24]:
e_pt =r'C:\Users\Anton\deeplearn_train\rti_1\rusvector'
opt_f=e_pt+r'\options.json'
weight_f=e_pt+r'\model.hdf5'

In [25]:
elmo=Elmo(opt_f,weight_f,1)

In [26]:
def prepared_title(ind):
    ls=[]
    for tk in df.title[ind].split():
        prs=morf.parse(tk)[0]
        if prs.tag.POS not in ['PREP','CONJ']:
            ls.append(prs.normal_form)
    return ls

In [27]:
elmo.eval();

In [33]:
elmo.cuda();

### Формируем тензор с резульатами пропуска заголовка через ELMo

In [39]:
%%time
n = df.shape[0]
b=torch.zeros(n,1025)
for i in range(n):
    t_inp=batch_to_ids([prepared_title(i)]).cuda()
    emb=elmo(t_inp)
    
    b[i,:1024]=emb['elmo_representations'][0].detach()[0].sum(dim=0).cpu()
    b[i,1024]=df.tit_len[i].astype(float)
    torch.cuda.empty_cache()
    if i%10000==0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Wall time: 1h 5min 55s


In [40]:
b.shape

torch.Size([100000, 1025])

##  Декларируем простую сеть для дообучения классификатора

In [50]:
text_tune = nn.Sequential(
            nn.Linear(1029, 500),
            nn.Dropout(0.2),
            nn.Linear(500, 500),
            nn.ReLU(),    
            nn.Linear(500, 22)
  
            )

In [99]:
text_tune.train();

In [13]:
# a - входной тензор
a=torch.zeros(100000,1029)
a[:,:1025]=b.clone()
a[:,1025:]=geo.clone()

In [16]:
target = torch.LongTensor(df.top_id)

In [100]:
bs = 1000
num_epochs = 100
optimizer = torch.optim.Adam(text_tune.parameters(), lr=0.000003)
criterion = nn.CrossEntropyLoss()

In [101]:
# тренирвока модели
l1=0
l2=60
l3=70
l4=80

for epoch in range(num_epochs):
    run_l=0

    for i in range(0,90000,bs): 
        inp = a[i:(i+bs),:].float()
        inp2 = target[i:(i+bs)] 
        optimizer.zero_grad()        
        out=text_tune(inp)

        loss = criterion(out,inp2)
        loss.backward()
        
        run_l+=loss.item()      
        optimizer.step()



    if epoch%5==0:
        l4=l3
        l3=l2          
        l2=l1
        l1=run_l

        if (l1>l2) & (l2>l3) & (l3>l4):
            print('end if trane')
            break
        text_tune.eval()
        out_test = text_tune(a[90000:,:].float()).cpu()
        tet_res=target[90000:].cpu()
        text_tune.train()
        val=f1_score(tet_res.numpy(),out_test.argmax(axis=1).numpy(),average='micro')
        res=text_tr()
        print(val,res,l1 )  
        if res-val>0.06:
            print('GAP is reached.end if trane')
            break

0.6915 0.7140444444444445 81.24198746681213
0.6923 0.7145333333333334 81.11734938621521
0.6924 0.7151222222222222 80.8876165151596
0.6923 0.7156555555555556 80.60405975580215
0.6925 0.7165111111111111 80.54951596260071
0.6934 0.7171444444444445 80.19935989379883
0.6934 0.7179222222222222 80.08700048923492
0.6932 0.7183555555555555 79.81382071971893
0.6936 0.7189444444444445 79.64801543951035
0.6943 0.719411111111111 79.37538051605225
0.6942 0.7200555555555556 79.19245648384094
0.694 0.7207 78.88361376523972
0.6951 0.721088888888889 78.68443942070007
0.6948 0.7213333333333334 78.54552394151688
0.6948 0.7218222222222221 78.4982972741127
0.6947 0.7224666666666667 78.16089129447937
0.6943 0.7229555555555556 78.09604912996292
0.6951 0.723688888888889 77.78736090660095
0.6955 0.7241777777777778 77.59200406074524
0.6962 0.7244000000000002 77.47793650627136


In [21]:
def text_tr():
    text_tune.eval()
    out_test = text_tune(a[:90000,:].float()).cpu()
    tet_res=target[:90000].cpu()
    text_tune.train()
    return(f1_score(tet_res.numpy(),out_test.argmax(axis=1).numpy(),average='micro') )  

## Запишем результаты предсказания модели в датасет

In [102]:
text_tune.eval();

out_test = text_tune(a.float())
df['pred_1']=out_test.argmax(axis=1).numpy()

df9=df.loc[90000:,:]

In [62]:
df9.head()

,url,title,text,topic,tags,date,top_id,tit_len,pred_1
90000,https://lenta.ru/news/2012/10/19/blame/,Тимошенко обвинила Януковича в туалетном вуайе...,"Экс-премьер Украины, лидер оппозиционной ""Бать...",Бывший СССР,Все,2012/10/19,4,6,4
90001,https://lenta.ru/news/2011/08/18/detain1/,Трех жителей Киргизии задержали за убийство ту...,МВД Киргизии объявило о раскрытии убийства Ерл...,Бывший СССР,Все,2011/08/18,4,7,0
90002,https://lenta.ru/news/2012/08/28/ministry/,Эстонские власти разработали концепцию гей-сож...,Эстонское министерство юстиции разработало кон...,Бывший СССР,Все,2012/08/28,4,5,4
90003,https://lenta.ru/news/2009/08/04/sabotage/,Грустная Тимошенко заподозрила Ющенко в попытк...,Премьер-министр Украины Юлия Тимошенко заподоз...,Бывший СССР,Все,2009/08/04,4,8,3
90004,https://lenta.ru/news/2016/06/02/peskov_kadyrov/,В Кремле прокомментировали сорвавшийся визит п...,Пресс-секретарь президента России Дмитрий Песк...,Россия,Политика,2016/06/02,0,8,0


## Снижаем количество признаков до 4. Для этого используем жадный алгоритм элиминации: Ищем самую слабую тему с наименьшим accuracy и затем объединяем ее с одной из оставшихся, выбирая (перебором) из них такую вторую тему, которая показывает дает наибольший прирост метрики F1 на всем датасете.

In [33]:
# функция считает F1 относительно групп тем. 
# группы формируется с помощью вспомогательного датасета tpk
def foo():
    ar1=np.array(df9[['top_id','pred_1']].merge(tpk[['id','new_id']],left_on='top_id',right_on='id',how='left').new_id)
    ar2=np.array(df9[['top_id','pred_1']].merge(tpk[['id','new_id']],left_on='pred_1',right_on='id',how='left').new_id)
    
    return f1_score(ar1.astype(int),ar2.astype(int),average='micro')

In [34]:
# расчет accuracy по одной теме

def foo_i(i):
    df1=df9[['top_id','pred_1']].merge(tpk[['id','new_id']],left_on='top_id',right_on='id',how='left')
    df2=df9[['top_id','pred_1']].merge(tpk[['id','new_id']],left_on='pred_1',right_on='id',how='left')
    
    df3=df1[['new_id']].merge(df2[['new_id']],left_index=True,right_index=True)
    df3['bingo']=df3.new_id_x-df3.new_id_y

    if df3[(df3.new_id_x==i)].shape[0]==0:
        acc=0
    else:
        acc = df3[(df3.new_id_x==i) & (df3.bingo==0)].shape[0]/df3[(df3.new_id_x==i)].shape[0]
    
    return acc

In [69]:
# вспомогательная ячейка чтобы сформировать tpk
df_count=df9.topic.value_counts().reset_index()
df_count.columns=['tp','nmb']
def nmb_val(tp):
    if df_count[df_count.tp==tp].shape[0]==0:
        return 0
    return df_count[df_count.tp==tp].nmb.iloc[0]

In [108]:
tpk = pd.DataFrame(columns=['id','top','nmb'])
for i in top_ls:
    tpk.loc[tpk.shape[0],:]=[top_ls.index(i),i,nmb_val(i)]
#     print(top_ls.index(i),i)
tpk['new_id']=tpk.id

# Вывод итоговых результатов. В колонке 'new_id' - указанна группа (одна из четырех) к которой принадлежит данная тема. Первое число - значение метрики F1

In [112]:
print(foo())
tpk

0.8532


,id,top,nmb,new_id
0,0,Россия,2141,2
1,1,Мир,1836,1
2,2,Экономика,1066,2
3,3,Спорт,847,3
4,4,Бывший СССР,740,4
5,5,Культура,727,2
6,6,Наука и техника,725,2
7,7,Интернет и СМИ,577,2
8,8,Из жизни,351,2
9,9,Дом,348,2


## Cама процедура элиминации

In [110]:
while tpk.new_id.nunique()>4:
# ищем самый минимальный accuracy
    min_acc = 1
    min_ind=0
    for i in tpk.new_id.unique():
        acc_i = foo_i(i)
        if acc_i<min_acc:
            min_acc=acc_i
            min_ind=i
#     print(min_ind,tpk.new_id.unique(),'-минималисимус')
            
#  нашли. теперь пытаемся его объединить с кем нибудь с наименьшими потерями
    max_foo=0
    max_j=min_ind
    for j in tpk.new_id.unique():
#         print(j,foo())
        if j!=min_ind:
            tpk.loc[min_ind,'new_id']=j
            jfoo=foo()
            if (max_foo<jfoo) & (j>1):
                max_foo=jfoo
                max_j=j 
            else:
                tpk.loc[min_ind,'new_id']=min_ind

        else:
            tpk.loc[min_ind,'new_id']=min_ind

                
    for i in tpk[tpk.new_id==min_ind].index:
        tpk.loc[i,'new_id']=max_j

    
    print(max_j,foo(),tpk.new_id.nunique())

2 0.6965 21
4 0.6968 20
5 0.6973 19
2 0.6974 18
2 0.6974 17
2 0.6974 16
2 0.6974 15
2 0.6974 14
2 0.7036 13
6 0.7079000000000001 12
5 0.7117 11
2 0.713 10
5 0.7219 9
5 0.7262999999999998 8
2 0.735 7
5 0.7523000000000001 6
2 0.7771 5
2 0.8532 4


# Смотрим сбалансированность классов

In [111]:
tpk.groupby('new_id').nmb.sum()

new_id
1    1836
2    6563
3     847
4     754
Name: nmb, dtype: int64

## Сохраняем результаты

In [113]:
df.to_csv('df_100000.csv')
torch.save(a, 'a_100000.pt') 
torch.save(text_tune,'text_tune_100000')
torch.save(elmo,'elmo_100000_85')
torch.save(geo,'geo_tz_100000.pt')